In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [3]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [4]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [6]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

Success!


In [7]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCosts.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [8]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [9]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

In [10]:
# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

In [11]:
# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Total_Cost_Produced'] = 0

In [12]:
""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Total_Cost_Produced', rawGoodsDF['TotalCost'].sum())
    # to be saved for lack of costing info, currently not used!!!
    noCostDF = rawGoodsDF[rawGoodsDF['TotalCost'].isnull()].copy() # ... it's so nice ...
    # what a waste, eh?

In [13]:
newbuilds

,Part,Qty,Date,buildIndex,Total_Purchase,Total_Cost_Produced
0,016-1632-10 r01,38,2017-05-15,-1,16389,-30283
1,016-452-10 r03,54,2017-05-15,-2,2494,-17593
2,016-1622-10 r02,35,2017-05-15,-3,837,-12269
3,016-1922-10 r02,40,2017-05-15,-4,162380,-194957
4,016-1009-10 r02,40,2017-05-15,-5,29202,-57554
5,016-131-10,50,2017-05-15,-6,22175,-25067
6,016-1877-10 r03,370,2017-05-15,-7,239866,-196702
7,016-143-10,30,2017-05-15,-8,10377,-17536
8,016-138-10,20,2017-05-15,-9,3795,-9437
9,016-1847-10 r01,10,2017-05-15,-10,5211,-6386


In [14]:
newbuilds.to_clipboard()

In [ ]:
# using this as temporary variable to iterate through later
imagBuild = -1

In [ ]:
thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()

In [ ]:
buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()

In [ ]:
rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']

In [ ]:
noCostDF = rawGoodsDF[rawGoodsDF['TotalCost'].isnull()].copy()

In [ ]:
rawGoodsDF['TotalCost'].sum()

In [ ]:
thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()

In [ ]:
purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()

In [ ]:
noCostDF = purchDF[purchDF['TotalCost'].isnull()].copy()

In [ ]:
totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()

In [ ]:
totPurchDF['TotalCost'].sum()

In [ ]:
totPurchDF

In [ ]:
newbuilds.set_value(0, 'Total_Purchase', totPurchDF['TotalCost'].sum())